# Validalab, s'informer en confiance

Dans ce **TP**, nous allons exploiter la base de données de **Validalab**. <br>
Une organisation à but non lucratif donc le but est d'aider les citoyens à mieux s'informer et à être acteurs de leur consommation d'informations.<br>
Elle a été initiée par **Jean-Marc Guerin**

![jmg.png](../data/images/jmg.png)

Plutôt que de faire du fact-checking, **Validalab** prend le parti d'informer les utilisateurs sur la source d'information. <br>
Ainsi, sur cette [application](http://app.validalab.fr/), on peut retrouver différentes informations agrégées sur les médias français.<br>
>Les données ont été scrappées sur différents dites et ingérés dans une base de données **Neo4j**. Le [dictionnaire des données](https://docs.google.com/spreadsheets/d/17iylS3y-xRVZLFMOuyz-o5Oy_MSlO_jvQoY8LBmH1LQ/edit#gid=1217348665) décrit les **types de données**, les **propriétés** et les **relations** du graphe de Validalab.

## Installation du Driver python de Neo4j 

In [38]:
!pip install neo4j

## Importation des packages 

In [1]:
from neo4j import GraphDatabase, basic_auth

## Informations de connexion à la base de données

In [2]:
password = "Ncl9ptdC3GzPHM8ba-_0NWRWSH7iHDVZYMPd_qWrES0"
uri = "neo4j+s://5074307c.databases.neo4j.io"
driver = GraphDatabase.driver(uri,auth=basic_auth("neo4j", password))

In [3]:
db = driver.session(database='neo4j')

### 1- Introduction
Pour exécuter une requête **CYPHER** via le driver python de Neo4j, on utilise la méthode `run` de l'objet `Session`, soit `db` dans notre cas.

Exemple : Lister les différents types de données

In [36]:
# Lister les différents type de données
results = db.run("""
MATCH (n) RETURN DISTINct labels(n)
""")
results.data()

[{'labels(n)': ['Option']},
 {'labels(n)': ['Question']},
 {'labels(n)': ['Tag']},
 {'labels(n)': ['Template']},
 {'labels(n)': ['Section']},
 {'labels(n)': ['Project']},
 {'labels(n)': ['TestNode']},
 {'labels(n)': ['Entity']},
 {'labels(n)': ['Website']},
 {'labels(n)': ['Facebook']},
 {'labels(n)': ['Wikipedia']}]

#### Help
Consultons l'aide de la méthode run

In [5]:
help(db.run)

Help on method run in module neo4j._sync.work.session:

run(query: 't.Union[te.LiteralString, Query]', parameters: 't.Optional[t.Dict[str, t.Any]]' = None, **kwargs: 't.Any') -> 'Result' method of neo4j._sync.work.session.Session instance
    Run a Cypher query within an auto-commit transaction.
    
    The query is sent and the result header received
    immediately but the :class:`neo4j.Result` content is
    fetched lazily as consumed by the client application.
    
    If a query is executed before a previous
    :class:`neo4j.Result` in the same :class:`.Session` has
    been fully consumed, the first result will be fully fetched
    and buffered. Note therefore that the generally recommended
    pattern of usage is to fully consume one result before
    executing a subsequent query. If two results need to be
    consumed in parallel, multiple :class:`.Session` objects
    can be used as an alternative to result buffering.
    
    For more usage details, see :meth:`.Transaction.

Elle a 2 principaux arguments(`query` et `parameters`) et des arguments indéfinis `**kwargs`.

- `query`: comme l'indique la docstring, c'est tout simplement la requête CYPHER
- `parameters`: il s'agit d'un dictionnaire de paramètres utilisables dans la requête précédente

In [35]:
# Afficher les informations sur l'entité pertant le nom "Le Monde SA"
results = db.run("""
    MATCH (n:Entity {name: $entity_name})
    RETURN n
    """, {"entity_name": "Le Monde SA"})
results.data()

[{'n': {'Diplo_milliardaireForbes': 'Non',
   'Diplo_mediaPeriodicite': 'Quotidien',
   'Diplo_typeLibelle': 'Media',
   'Diplo_mediaEchelle': 'National',
   'name': 'Le Monde SA',
   'Diplo_typeCode': 1,
   'Diplo_ACPMdiffusion': 100000,
   'Diplo_rangChallenges': 10,
   'Diplo_mediaType': 'Journal',
   'Diplo_commentaire': 'Commentaire'}},
 {'n': {'Diplo_typeLibelle': 'Personne Morale', 'name': 'Le Monde SA'}}]

### 2- Clause MATCH

Combien de nœuds de type **Entity**, **Website**, **Wikipedia** y a-t-il dans la base de données ?
> Créer une fonction permettant de compter le nombre de noeuds pour un Label donné.

**Lien utile :** https://neo4j.com/docs/cypher-manual/current/clauses/match/#basic-node-finding

In [37]:
def count_nodes(entity_type):
    results = db.run(f"""
    MATCH (n:{entity_type}) RETURN count(*) as count
    """)
    return results.data()[0].get('count')

In [38]:
for label in ["Entity", "Website", "Wikipedia"]:
    print(f"There is {count_nodes(label)} nodes with the type {label}")

There is 27 nodes with the type Entity
There is 9 nodes with the type Website
There is 9 nodes with the type Wikipedia


---
Lister les 10 premiers nœuds de type **Entity**

In [39]:
def find_nodes(entity_type,limit=10):
    results = db.run(f"""
    MATCH (n:{entity_type}) RETURN n LIMIT {limit}
    """)
    return results.data()

In [40]:
find_nodes("Entity", limit=1)

[{'n': {'name': 'AFP', 'type': 'Neutre'}}]

---
Afficher les nœuds de type Entity sous forme de **DataFrame**<br>
**N.B:** la méthode `to_df()` peut aider.

In [41]:
results = db.run("""
MATCH (n:Entity) RETURN n.Diplo_mediaPeriodicite, n.name, n.Diplo_typeLibelle LIMIT 10
""")
results.to_df()

,n.Diplo_mediaPeriodicite,n.name,n.Diplo_typeLibelle
0,None,AFP,None
1,None,Le Figaro,None
2,None,Agence France-Presse,None
3,None,French Government,Etat
4,Quotidien,Le Figaro,Media
5,None,FactCheckFictif,None
6,None,FactCheckFictif Org,None
7,None,Independent,Personne Morale
8,None,LeGorafi,None
9,None,Dassault Group,Personne Morale


### 3- Clause WHERE

Créer une fonction qui permet de retrouver des sites contenant une chaine de caractères.<br>
**Exemple :** lemonde, valeurs, etc...
<br>
**Lien utile :** https://neo4j.com/docs/cypher-manual/current/clauses/where/

In [42]:
def find_website(string):
    results = db.run(f"""
    MATCH (w:Website)
    WHERE w.name =~"(?i).*{string}.*" 
    RETURN w
    """)
    return results.data()

In [43]:
find_website('lemonde')

[{'w': {'name': 'lemonde.fr', 'url': 'https://lemonde.fr'}}]

### 4- RELATIONS

---
Créer une fonction qui retourne le résumé **Wikipedia** pour un site web donné.
> Rappel: Dans la base de données, nous avons les données Wikipedia. Le résumé des noeuds Wikipedia correspond à la propriété ``summary`.

In [44]:
def get_summary(site_name):
    results = db.run("""
    MATCH (n {name: $site_name})-[:OWNED_BY]->(e:Entity)<-[:OWNED_BY]-(w:Wikipedia)
    RETURN w.summary
    """, {"site_name":site_name})
    return results.data()[0].get('w.summary')

In [45]:
get_summary('lemessager.fr')

'LeMessager est un journal Français'

---
Le Gorafi, Le Monde et Valeurs Actuelles sont-ils des journaux fiables ?
Pour répondre à cette question, on peut lister les **citations** de ces médias.<br>
> Les `citations` d'un site, dans notre contexte,  sont les recommandations(<span style="background:green; color:white">positives</span> ou <span style="background:red; color:white">négative</span>) de ce site par des entités quelconques.

In [50]:
def list_recommendations(site_name):
    results = db.run("""
    MATCH(w:Website{name:$site_name})-[:OWNED_BY]->(n:Entity)<-[reco:RECOMMENDS]-(r) 
    RETURN r.name as recommender, reco.weight as weight, 
    reco.meaning as meaning, reco.sourceURL as sourceURL
    """, {"site_name":site_name})
    return results.to_df()

In [51]:
list_recommendations("lemonde.fr")

,recommender,weight,meaning,sourceURL
0,FactCheckFictif,9,Très Fiable,None
1,Le Monde Diplomatique,8,Fiable,None


In [52]:
list_recommendations("legorafi.fr")

,recommender,weight,meaning,sourceURL
0,Le Monde Diplomatique,2,Satirique,None


In [53]:
list_recommendations("valeursactuelles.com")

,recommender,weight,meaning,sourceURL
0,AFP,3,Biaisé,None


### 5- Investigations

---
Créer une fonction pour déterminer les propriétaires finaux d'un site web.
> Les propriétairs finaux sont ceux au sommet de la chaine. C'est à dire, ceux qui n'ont personne qui les possède.

In [54]:
def final_onwers(site_name):
    results = db.run("MATCH (w:Website) "
                "WHERE w.name =~ $site_name "
                "MATCH (w)-[:OWNED_BY*]->(e:Entity)"
                "MATCH (e)<-[:OWNED_BY*]-(n) "
                "WITH e, n "
                "WHERE NOT (e)-[:OWNED_BY]->()"
                "RETURN DISTINCT(e.name) as proprietaire",
                {"site_name": f"(?i).*{site_name}.*"})
    return [proprio.get('proprietaire') for proprio in results.data()]

In [55]:
final_onwers('lemessager.fr')

['Groupe Régional Presse']

---
Créer une fonction pour déterminer le nombre de médias que possèdent chacun des propriétaires finaux trouvés.

In [56]:
def medias_by_owners(site_name):
    results = db.run("MATCH (w:Website) "
                "WHERE w.name =~ $site_name "
                "MATCH (w)-[:OWNED_BY*]->(e:Entity)"
                "MATCH (e)<-[:OWNED_BY*]-(n) "
                "WITH e, n "
                "WHERE NOT (e)-[:OWNED_BY]->()"
                "RETURN DISTINCT e.name as proprietaire, count(n) as nb_medias",
                {"site_name": f"(?i).*{site_name}.*"})
    return results.to_df().sort_values(by="nb_medias", ascending=False)

In [57]:
medias_by_owners('lemonde.fr')

,proprietaire,nb_medias
0,Xavier Niel,4
